In [11]:
import SimpleITK as sitk
import pandas as pd

from pathlib import Path
from damply import dirs
from joblib import Parallel, delayed

from readii.image_processing import flattenImage, alignImages
from readii.io.loaders import loadImageDatasetConfig
from readii.io.writers.nifti_writer import NIFTIWriter, NiftiWriterIOError
from readii.negative_controls_refactor import NegativeControlManager
from readii.process.config import get_full_data_name
from readii.utils import logger

In [3]:
def get_readii_settings(dataset_config: dict) -> tuple[list, list, list]:
    """Extract READII settings from a configuration dictionary.
    
    Parameters
    ----------
    dataset_config : dict
        Configuration dictionary read in with `loadImageDatasetConfig` containing READII settings
    
    Returns
    -------
    tuple
        A tuple containing:
        - regions: list of regions to process
        - permutations: list of permutations to apply
        - crop: list of crop settings
    """
    readii_config = dataset_config['READII']
    if 'IMAGE_TYPES' not in readii_config:
        message = "READII configuration must contain 'IMAGE_TYPES'."
        logger.error(message)
        raise KeyError(message)
    
    regions = readii_config['IMAGE_TYPES']['regions']

    permutations = readii_config['IMAGE_TYPES']['permutations']

    crop = readii_config['IMAGE_TYPES']['crop']

    return regions, permutations, crop

In [4]:
def save_out_negative_controls(nifti_writer: NIFTIWriter,
                            patient_id: str,
                            image: sitk.Image,
                            region: str,
                            permutation: str):
    """Save out negative control images using the NIFTIWriter."""

    try:
        nifti_writer.save(
                        image,
                        PatientID=patient_id,
                        region=region,
                        permutation=permutation
                    )
    except NiftiWriterIOError as e:
        message = f"{permutation} {region} negative control file already exists for {patient_id}. If you wish to overwrite, set overwrite to true in the NIFTIWriter."
        logger.debug(message)

    return image

In [12]:
dataset = "NSCLC-Radiomics_test"
random_seed = 10

if dataset is None:
    message = "Dataset name must be provided."
    logger.error(message)
    raise ValueError(message)

config_dir_path = dirs.CONFIG / 'datasets'

dataset_config = loadImageDatasetConfig(dataset, config_dir_path)

dataset_name = dataset_config['DATASET_NAME']
full_data_name = get_full_data_name(config_dir_path / dataset)
logger.info(f"Creating negative controls for dataset: {dataset_name}")

# Extract READII settings
regions, permutations, _crop = get_readii_settings(dataset_config)

# Set up negative control manager with settings from config
manager = NegativeControlManager.from_strings(
    negative_control_types=permutations,
    region_types=regions,
    random_seed=random_seed
)

mit_images_dir_path = dirs.PROCDATA / full_data_name / 'images' /f'mit_{dataset_name}'

dataset_index = pd.read_csv(Path(mit_images_dir_path, f'mit_{dataset_name}_index.csv'))

# StudyInstanceUID
for study, study_data in dataset_index.groupby('StudyInstanceUID'):
    print(f"Processing StudyInstanceUID: {study}")

    # Get image metadata as a pd.Series
    image_metadata = study_data[study_data['Modality'] == 'CT'].squeeze()
    image_path = Path(image_metadata['filepath'])
    # Load in image
    raw_image = sitk.ReadImage(mit_images_dir_path / image_path)
    # Remove extra dimension of image, set origin, spacing, direction to original
    image = alignImages(raw_image, flattenImage(raw_image))

    # Get mask metadata as a pd.Series
    mask_metadata = study_data[study_data['Modality'] == 'RTSTRUCT'].squeeze()
    mask_path = Path(mask_metadata['filepath'])
    # Load in mask
    raw_mask = sitk.ReadImage(mit_images_dir_path / mask_path)
    mask = alignImages(raw_mask, flattenImage(raw_mask))

    # Set up writer for saving out the negative controls
    nifti_writer = NIFTIWriter(
        root_directory = mit_images_dir_path.parent / f'readii_{dataset_name}' / image_path.parent,
        filename_format = "{permutation}_{region}.nii.gz",
        overwrite = True,
        create_dirs = True
    )

    for neg_image, permutation, region in manager.apply(image, mask):
        image = save_out_negative_controls(
                nifti_writer, 
                patient_id = image_metadata['PatientID'],
                image = neg_image,
                region = region,
                permutation = permutation
            )


Processing StudyInstanceUID: 1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095


15:07:14 [warning  ] File /home/bhkuser1/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-Radiomics_test/LUNG1-002_0001/CT_23261228/sampled_roi.nii.gz already exists. Overwriting. [readii] call=nifti_writer.save:108


Processing StudyInstanceUID: 1.3.6.1.4.1.32722.99.99.239341353911714368772597187099978969331


15:07:17 [warning  ] File /home/bhkuser1/katy/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics_test/images/readii_NSCLC-Radiomics_test/LUNG1-001_0000/CT_63382046/sampled_roi.nii.gz already exists. Overwriting. [readii] call=nifti_writer.save:108


In [6]:
image_metadata['filepath']

'LUNG1-001_0000/CT_63382046/CT.nii.gz'

In [9]:
image.GetSpacing()

(1.0, 1.0, 1.0)

In [10]:
raw_image = sitk.ReadImage(mit_images_dir_path / image_path)
raw_image.GetSpacing()

(0.9765625, 0.9765625, 3.0)

In [14]:
neg_image.GetSpacing()

(0.9765625, 0.9765625, 3.0)